# Load CISI.ALL and put data into lists

In [1]:
with open('CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")


cisi_indecies = []
titles = []
corpus = []


for l in lines:
    indicator = l[1:2]
    content = l[3:]
    if indicator == 'W':
        corpus.append(content)
    elif indicator == 'I':
        cisi_indecies.append(content)
    elif indicator == 'T':
        titles.append(content)

# Load CISI.QRY

In [2]:
with open('CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

queries = []    

for l in lines:
    indicator = l[1:2]
    content = l[3:]
    if l.startswith(".W"):
        queries.append(content)
        
queries[0]

'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import numpy as np
import os

def same_len(corpus, titles):
    if len(new_corpus) != len(new_titles):
        print("not for each doc a title was given")
        print("number of docs:", len(corpus))
        print("number of titels:", len(titles))
        return False
    else:
        return True
    

class tfidf:
    def __init__(self, corpus, titles):
        
        self.tfidfVectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
        self.corpus = corpus
        self.titles = titles
        self.tfidf_mat = self.tfidfVectorizer.fit_transform(self.corpus)
        
    def add_to_corpus(self, new_corpus:list, new_titles:list):
        
        self.corpus += new_corpus
        self.titles += new_titels
        
        self.tfidf_mat = self.tfidfVectorizer.fit_transform(self.corpus)
    
    # returns a list of indicies sorted by relevance
    def query_indicies(self, query: str):
        
        voc = self.tfidfVectorizer.get_feature_names()
        tfidfvectorizer_query = TfidfVectorizer(analyzer='word',stop_words= 'english',vocabulary=voc)
        
        query_tfidf = tfidfvectorizer_query.fit_transform([query])
        
        # matrix multiplicatin to calculate cosine similarity
        query_result_matrix = self.tfidf_mat @ query_tfidf.reshape(-1,1)
        
        # result_list will be filled with (index, idf-value) tuples
        result_list =[]
        
        for i in range(query_result_matrix.shape[0]):
            result_list.append((i,query_result_matrix[i,0]))
        
        #sorts from large to small based on idf-value
        result_list.sort(key=lambda x:x[1], reverse=True)
        
        only_indices = []
        for i in range(len(result_list)):
            only_indices.append(result_list[i][0])

        return only_indices
    
    def query_titles(self, query:str):
        
        indicies = self.query_indicies(query)
        titles = []
        for i in indicies:
            titles.append(self.titles[i])
        
        return titles
    
    def query_k_titles(self, query:str, k=10):
        
        return self.query_titles(query)[:k]

In [10]:
n = 17
tfidf_cisi = tfidf(corpus,titles)
queries[n], tfidf_cisi.query_k_titles(queries[n])

('What methods are there for encoding, automatically matching, and automatically drawing structures extended in two dimensions, like the structural formulas for chemical compounds?',
 ['The Automatic Encoding of Chemical Structures',
  'Installation and Operation of a Registry for Chemical Compounds',
  'Automatic Document Classification   Part II. Additional Experiments',
  'Encoding and Decoding WLN',
  'The Wiswesser Line-Formula Chemical Notation (WLN)',
  'The IDC System for Chemical Documentation',
  'A Business Intelligence System',
  'French National Policy for Chemical Information and the DARC System as a Potential Tool of This Policy',
  'Experience with the Mechanized Chemical and Biological Information Retrieval System',
  'Computer Indexing of Medical Articles - Project Medico'])

In [13]:
all_res = []
for q in queries:
    all_res.append((q, tfidf_cisi.query_k_titles(q)))
    
all_res

[('What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?',
  ['Information Transfer Limitations of Titles of Chemical Documents',
   'Are Titles of Chemical Papers Becoming More Informative?',
   'The Information Content of Titles in Engineering Literature',
   'Comparative Efficiency of Searching Titles, Abstracts, and Terms in a Free-Text Data Base',
   'Current Physics Information',
   'Relative Effectiveness of Document Titles and Abstracts for Determining Relevance of Documents',
   'Selected Lists of Journals for the Small Medical Library: A Comparative Analysis',
   'Relative Effectiveness of Titles, Abstracts, and Subject Headings for Machine Retrieval from the COMPENDEX Services',
   'A New Look at Reference Scattering',
   'A Study of Searching the Eye Research Literature']),
 ('How can actually pe

In [14]:
text = "this is a search for documents"
text, tfidf_cisi.query_k_titles(text)

('this is a search for documents',
 ['Evaluation of Search Time for Two Computerized Information Retrieval Systems at the University of Georgia',
  'Reader in Documents of International Organizations',
  'A Literature Search and File Organization Model',
  'Evaluation of Information Loss Probability in Indexing',
  'The SMART Retrieval System Experiments in Automatic Document Processing',
  'Information-Seeking Behavior of Catalog Users',
  'Searching CA Condensates On-Line vs. the CA Keyword Indexes',
  'The SMART Automatic Document Retrieval System - An Illustration',
  'A Comparison of a Keyword from Title Index with a Single Access Point per Document Alphabetic Subject Index',
  'Modern Integral Information Systems for Chemistry and Chemical Technology'])